<a href="https://colab.research.google.com/github/leandrosfarias/analisador_lexico/blob/regex/analisador_lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### modulo.erro

In [20]:
import re

class Erro():

  def __init__(self):
    pass


  def checa_linguagem(programa):
    linhas = programa
    une_linhas = ''
    for linha in programa:
      une_linhas += re.sub('\/\/.*$', '', linha.strip().replace(' ', ''))
    so_simbolos = re.sub('[a-zA-Z]+|\".+[^"]\"|\d+', "", une_linhas)
    regra_simbolos_diferentes = '[^\:=|;|\.|,|\(|\)|\:|\+|\-|\*|\/|<=|>=|\==|\<>|<|>|\~]'
    if bool(re.search(regra_simbolos_diferentes, so_simbolos)):
      simbolos_diferentes = re.findall(regra_simbolos_diferentes, so_simbolos)
      for linha in linhas:
        if simbolos_diferentes[0] in linha:
          raise Exception(f'erro encontrado na linha: {linhas.index(linha)+1}')


### modulo.linguagem

In [21]:
class Gramatica():
  
  regras = {
      'strings':                '\".+[^"]\"',
      'palavras_reservadas':    'PROGRAM|INTEGER|BOOLEAN|BEGIN|END|WHILE|DO|READ|VAR|FALSE|TRUE|WRITE|IF|ELSE|THEN',
      'simbolos':               '\:=|;|\.|,|\(|\)|\:',
      'operadores_aritmeticos': '\+|\*|\/',
      'menos':                  '\-',
      'operador_negacao':       '\~',
      'identificadores':        '[a-zA-Z]+[0-9]*',
      'numeros':                '[0-9]+',
      'operadores_relacionais': '<=|>=|\==|\<>|<|>',
      'operadores_logicos':     'OR|AND',
  }


  def __init__(self):
      self.gramatica = self.une_regras()


  def une_regras(self):
      regex = ''
      for regra in self.regras.values():
          regex += f'{regra}|'
      regex = regex[:-1]
      return regex


### modulo.scanner

In [22]:
import re

class Scanner():
  
  def __init__(self, caminho):
      self.programa = open(caminho).read()
      linhas_programa = open(caminho).readlines()
      Erro.checa_linguagem(linhas_programa)
      self.trata_programa()


  def trata_programa(self):
    self.programa = self.programa.split('\n')
    for linha in self.programa:
      self.programa[self.programa.index(linha)] = linha.strip()
      if '//' in linha:
        if linha.startswith('//'):
          self.programa.remove(linha)
        else:
          self.programa[self.programa.index(linha)] = re.sub('\/\/.*$', '', linha).strip()
             

### modulo.token

In [23]:
class Token():
  
  tipos = {
  'ID':      ["ID", ""],
  'NUMERO':  ["NUMERO", ""],
  '+':       ["OPAD", "MAIS"],
  '-':       "MENOS",
  '*':       ["OPMULT", "VEZES"],
  '/':       ["OPMULT", "DIV"],
  'OR':      ["OPLOG", "OR"],
  'AND':     ["OPLOG", "AND"],
  '~':       ["OPNEG", "NEG"],
  '<':       ["OPREL", "MENOR"],
  '<=':      ["OPREL", "MENIG"],
  '>':       ["OPREL", "MAIOR"],
  '>=':      ["OPREL", "MAIG"],
  '==':      ["OPREL", "IGUAL"],
  '<>':      ["OPREL", "DIFER"],
  ';':       "PVIG",
  '.':       "PONTO",
  ':':       "DPONTOS",
  ',':       "VIG",
  '(':       "ABPAR",
  ')':       "FEPAR",
  ':=':      "ATRIB",
  'PROGRAM': "PROGRAM",
  'INTEGER': "INTEGER",
  'BOOLEAN': "BOOLEAN",
  'BEGIN':   "BEGIN",
  'END':     "END",
  'WHILE':   "WHILE",
  'DO':      "DO",
  'READ':    "READ",
  'VAR':     "VAR",
  'FALSE':   "FALSE",
  'TRUE':    "TRUE",
  'WRITE':   "WRITE",
  'IF':      "IF",
  'ELSE':    "ELSE",
  'THEN':    "THEN",
  'STRING':  "CADEIA"
  }


  def __init__(self, caracter):
      self.caracter = caracter
      self.tipo = self.define_tipo()


  def define_tipo(self):
    if self.caracter in Token.tipos.keys():
      if self.caracter == 'ID' or self.caracter == 'STRING' or self.caracter == 'NUMERO':
        token_aux = Token.tipos['ID']
        token_aux[1] = self.caracter
        return token_aux
      else:
        for simbolo in Token.tipos.keys():
            if self.caracter == simbolo:
                return Token.tipos[simbolo]  
    elif self.caracter[0] == '"' and self.caracter[-1] == '"':
        return Token.tipos['STRING']
    elif self.caracter.isnumeric():
      token_aux = Token.tipos['NUMERO']
      token_aux[1] = self.caracter
      return token_aux
    else:
      if len(self.caracter) > 16:
        self.caracter = self.caracter[:16]
        token_aux = Token.tipos['ID']
        token_aux[1] = self.caracter
        return token_aux
      else:
        token_aux = Token.tipos['ID']
        token_aux[1] = self.caracter
        return token_aux
          

  def imprimir(self):
      if type(self.tipo) == list:
          return f'<{self.tipo[0]}, {self.tipo[1]}>'
      else:
          return f'<{str(self.tipo)}>'


In [24]:

class Tokenizador():

  def __init__(self, programa):
    self.programa = programa

    
  def retorna_tokens(self):
    scanner = Scanner(self.programa)
    linguagem = Gramatica().gramatica

    tokens_nativos = []

    for linha in scanner.programa:
      tokens_nativos.append(re.findall(linguagem, linha))
    
    for grupo_tokens in tokens_nativos:
      tokens_tipados = ''
      for token in grupo_tokens:
        tokens_tipados += f'{Token(token).imprimir()} '
      print(tokens_tipados)


### main

In [ ]:
Tokenizador('teste.txt').retorna_tokens()

teste.txt

PROGRAM example;
VAR x : INTEGER; //rtreyyrty
BEGIN
    x := 5 * 10;
    WRITE(x);
END. "ret345?/:;>,]}[{~´_=+|\!@#$%&*()"